In [1]:
import pandas as pd

bhp = pd.read_csv('https://raw.githubusercontent.com/kgpark88/ai-summary/main/BostonHousing.csv')

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(bhp.drop('MEDV', axis=1), bhp['MEDV'], test_size=0.15, random_state=2021, stratify = bhp['CHAS'])

In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=15/(70+15),
                                                      random_state=2021, stratify = x_train['CHAS'])

In [7]:
# MinMax스케일링
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
x_train[['NOX', 'RM']] = mm_scaler.fit_transform(x_train[['NOX', 'RM']] )
x_valid[['NOX', 'RM']] = mm_scaler.transform(x_valid[['NOX', 'RM']] )
x_test[['NOX', 'RM']] = mm_scaler.transform(x_test[['NOX', 'RM']] )

In [8]:
# 표준화스케일링
from sklearn.preprocessing import StandardScaler

st_scaler = StandardScaler()
x_train[['RAD', 'B']] = st_scaler.fit_transform(x_train[['RAD', 'B']] )
x_valid[['RAD', 'B']] = st_scaler.transform(x_valid[['RAD', 'B']] )
x_test[['RAD', 'B']] = st_scaler.transform(x_test[['RAD', 'B']] )

In [12]:
# 선형회귀 모델
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(x_train, y_train)

In [13]:
# ridge regression 모델
from sklearn.linear_model import Ridge
ridge = Ridge(5).fit(x_train, y_train)

In [14]:
# lasso regression 모델
from sklearn.linear_model import Lasso
lasso = Lasso(1).fit(x_train, y_train)

In [15]:
# elastic net regression 모델
from sklearn.linear_model import ElasticNet
elastic = ElasticNet(alpha=3.5, l1_ratio=(0.5/3.5)).fit(x_train, y_train)

In [16]:
# 모델의 성능
from sklearn.metrics import mean_squared_error as MSE

model_names = ['선형회귀', 'Ridge', 'Lasso', 'Elastic Net']
models = [lr, ridge, lasso, elastic]

for i in range(4) :
    y_pred = models[i].predict(x_valid)
    print(f"RMSE of {model_names[i]} : {MSE(y_valid, y_pred)**.5:.2f} * $1000")

RMSE of 선형회귀 : 4.96 * $1000
RMSE of Ridge : 5.46 * $1000
RMSE of Lasso : 6.46 * $1000
RMSE of Elastic Net : 6.74 * $1000


In [17]:
from sklearn.metrics import mean_absolute_error as MAE

model_names = ['선형회귀', 'Ridge', 'Lasso', 'Elastic Net']
models = [lr, ridge, lasso, elastic]

for i in range(4) :
    y_pred = models[i].predict(x_test)
    print(f"RMSE of {model_names[i]} : {MAE(y_test, y_pred):.2f} * $1000")

RMSE of 선형회귀 : 3.29 * $1000
RMSE of Ridge : 3.36 * $1000
RMSE of Lasso : 3.74 * $1000
RMSE of Elastic Net : 3.78 * $1000


In [18]:
# 모델의 성능을 출력
from sklearn.metrics import mean_absolute_error as MAE

model_names = ['선형회귀', 'Ridge', 'Lasso', 'Elastic Net']
models = [lr, ridge, lasso, elastic]

cond_cha =  (x_test['CHAS'] == 1)
x_test_cha = x_test[cond_cha]
y_test_cha = y_test[cond_cha]

for i in range(4) :
    y_pred_cha = models[i].predict(x_test_cha)
    print(f"RMSE of {model_names[i]} : {MAE(y_test_cha, y_pred_cha):.2f} * $1000")

RMSE of 선형회귀 : 5.32 * $1000
RMSE of Ridge : 5.86 * $1000
RMSE of Lasso : 5.66 * $1000
RMSE of Elastic Net : 5.62 * $1000


In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [20]:
keras.backend.clear_session()

model = Sequential()
model.add(Dense(32, activation='swish', input_shape=(13,)))
model.add(BatchNormalization())
model.add(Dense(32, activation='swish'))
model.add(BatchNormalization())
model.add(Dense(1))

model.compile(optimizer='adam',
              loss='mse'  ) 

mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
history = model.fit(x_train, y_train, epochs=200, batch_size=32,
                    verbose=0,validation_data=(x_valid, y_valid),callbacks=[mc])

In [22]:
# import matplotlib.pyplot as plt

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Loss')
# plt.xlabel('epochs')
# plt.ylabel('MSE')
# plt.legend(['loss', 'val_loss'])
# plt.show()